# Preprocessing and geometry in a feature space

This coursework is designed to practically explore the content given in the lectures in week 3.

## Hints

- This one is a lot more challenging than last week so don't panic, break everything down into small steps

- For the further work questions it may be helpful to write your code using functions rather than scripts (or even classes if they are familiar to you)

In [1]:
import numpy as np
#DO NOT CHANGE THIS CODE
X_train=np.loadtxt('./data/X_train_linear.txt')
X_test=np.loadtxt('./data/X_test_linear.txt')
y_train=np.loadtxt('./data/y_train_linear.txt')
y_test=np.loadtxt('./data/y_test_linear.txt')

## 1) Geometry in a kernel-defined feature space

### a) Compute the linear kernel of the linear training data without centering the data and calculate the norm of the mean of a sample

In [2]:
#YOUR CODE HERE

### b) Calculate the expected squared distance to the mean of a sample

In [3]:
#YOUR CODE HERE

### c) By subtracting the appropriate matrix, implement the centering of this kernel matrix and verify that the norm of the mean of a sample is zero

In [4]:
#YOUR CODE HERE

### d) Verify that the expected squared distance to the mean of a sample with the centred kernel is the same as the uncentred kernel

In [5]:
#YOUR CODE HERE

### e) By centering the original data and computing the linear kernel representation of the centered data show that the kernel representations are equivalent

In [6]:
#YOUR CODE HERE

## 2) Simple Classification Algorithm
In this part we will build the simple classifier described in the lectures

In [7]:
import numpy as np
#DO NOT CHANGE THIS CODE
X_train=np.loadtxt('./data/X_train_poly.txt')
X_test=np.loadtxt('./data/X_test_poly.txt')
y_train=np.loadtxt('./data/y_train_poly.txt')
y_test=np.loadtxt('./data/y_test_poly.txt')

### a) Write a function which calculates the mean of the kernel function of a sample with the positive and negative training data (i.e. the terms that aren't b)

In [8]:
#YOUR CODE HERE

### b) Write a function which calculates the b term in the simple classifier

In [9]:
#YOUR CODE HERE

### c) Write the decision function of the classifier

In [10]:
#YOUR CODE HERE

### c) Calculate the accuracy of this simple classifier

In [11]:
#YOUR CODE HERE

## 3) Fisher Discriminant Analysis

### a) Write a function which calculates the matrices B, C, and D

In [12]:
#YOUR CODE HERE

### b) Write a function which calculates the dual variables for the FDA classifier

In [13]:
#YOUR CODE HERE

### c) Write a function which calculates t and therefore the offset term b

In [14]:
#YOUR CODE HERE

### d) Write the decision function of the classifier

In [15]:
#YOUR CODE HERE

## 4) Further Investigation

### a) For one or both classifiers Demonstrate how the performance of the classifier changes if one class is overrepresented in the training data as compared to the test data (i.e. there are imbalanced classes)

### b) Demonstrate how regularisation can be used to improve the generalisation of the FDA classifier